In [ ]:
import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import glob
import indices_function as ifun
import warnings
from rechunker import rechunk
import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.colors as mcolors

warnings.filterwarnings('ignore')

In [ ]:
cluster = LocalCluster(
    n_workers=10, 
    threads_per_worker=1,
    timeout='3600s',
    memory_limit='5GB',   
)
client = Client(cluster)
client

In [ ]:
var="pr"
DCP_base=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/annual_average/"+var+"/2014.zarr")

datasets=[]
models=glob.glob("/nobackupp28/skhajehe/dcp-indices/multimodel/quantile/"+var+"/*.zarr")
# List of file names
for model in models:
    if any(x in model for x in ["one","test"]):
        print(model)
        continue
    X1=xr.open_zarr(model,consolidated=False)
    datasets.append(X1)
DCP = xr.concat(datasets, dim='model').mean(dim="model")
DCP=DCP.where(DCP_base[var].notnull())

PRISM=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/prism/quantile/"+var+"/quantiles.zarr")
PRISM=PRISM.where(DCP_base[var].notnull())

DCP['lon'] = (DCP['lon'] + 180) % 360 - 180
DCP = DCP.sortby(DCP.lon)
PRISM['lon'] = (PRISM['lon'] + 180) % 360 - 180
PRISM = PRISM.sortby(PRISM.lon)
DCP.load()
PRISM.load()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Define color intervals and corresponding colors
intervals = np.arange(0, 24,2)
colors = plt.cm.viridis_r(np.linspace(0, 1, len(intervals) - 1))  # Use YlOrRd colormap

# Create a custom colormap with specified intervals and colors
cmap = mcolors.ListedColormap(colors)
bounds = mcolors.BoundaryNorm(intervals, cmap.N)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=20)

plot = DCP.pr.sel(quantile=0.75).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' ,add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('DCP30 Ensemble Mean 75th Percentile Precipitation [mm/day]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

# cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
# cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
# cbar.set_label('Precipitation [mm/day]',fontweight='bold', fontsize=14)
# cbar.ax.yaxis.set_tick_params(labelsize='large')
# plt.show()
plt.savefig('./plot/fig3_0.75_precip_dcp.png')


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=20)

plot = PRISM.pr.sel(quantile=0.75).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' ,add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('PRISM 75th Percentile Precipitation [mm/day]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

# cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
# cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
# cbar.set_label('Precipitation [mm/day]',fontweight='bold', fontsize=14)
# cbar.ax.yaxis.set_tick_params(labelsize='large')
# plt.show()
plt.savefig('./plot/fig3_0.75_precip_prism.png')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Define color intervals and corresponding colors
intervals = np.arange(-5, 5, 0.5)
colors = plt.cm.PuOr(np.linspace(0, 1, len(intervals) - 1))  # Use YlOrRd colormap

# Create a custom colormap with specified intervals and colors
cmap = mcolors.ListedColormap(colors)
bounds = mcolors.BoundaryNorm(intervals, cmap.N)



In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-5, vcenter=0, vmax=1)

plot = (((DCP-PRISM)/PRISM)*100).pr.sel(quantile=0.25).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' , add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('25th Percentile Precipitation Relative Difference', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
cbar.set_label('Precipitation Relative Difference',fontweight='bold', fontsize=14)
plt.savefig('./plot/fig3_0.25_precip_rel_diff.png')


In [ ]:
var="tasmax"
DCP_base=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/annual_average/"+var+"/2014.zarr")

datasets=[]
models=glob.glob("/nobackupp28/skhajehe/dcp-indices/multimodel/quantile/"+var+"/*.zarr")
# List of file names
for model in models:
    X1=xr.open_zarr(model,consolidated=False)
    datasets.append(X1)
DCP = xr.concat(datasets, dim='model').mean(dim="model")
DCP=DCP.where(DCP_base[var].notnull())

PRISM=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/prism/quantile/"+var+"/quantiles.zarr")
PRISM=PRISM.where(DCP_base[var].notnull())

DCP['lon'] = (DCP['lon'] + 180) % 360 - 180
DCP = DCP.sortby(DCP.lon)
PRISM['lon'] = (PRISM['lon'] + 180) % 360 - 180
PRISM = PRISM.sortby(PRISM.lon)
DCP.load()
PRISM.load()

In [ ]:
DCP

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Define color intervals and corresponding colors
intervals = np.arange(-10, 40, 4)
colors = plt.cm.inferno(np.linspace(0, 1, len(intervals) - 1))  # Use YlOrRd colormap

# Create a custom colormap with specified intervals and colors
cmap = mcolors.ListedColormap(colors)
bounds = mcolors.BoundaryNorm(intervals, cmap.N)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=20)

plot = DCP.tasmax.sel(quantile=0.75).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' ,add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('DCP30 Ensemble Mean 75th Percentile Max Temperature [Degree C]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

# cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
# cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
# cbar.set_label('Temperature [Degree C]',fontweight='bold', fontsize=14)
# cbar.ax.yaxis.set_tick_params(labelsize='large')
# plt.show()
plt.savefig('./plot/fig2_0.75_max_temp_dcp.png')


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=20)

plot = PRISM.tasmax.sel(quantile=0.75).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' ,add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('PRISM 75th Percentile Max Temperature [Degree C]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

# cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
# cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
# cbar.set_label('Temperature [Degree C]',fontweight='bold', fontsize=14)
# cbar.ax.yaxis.set_tick_params(labelsize='large')
# plt.show()
plt.savefig('./plot/fig2_0.75_max_temp_prism.png')


In [ ]:
## import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Define color intervals and corresponding colors
intervals = np.arange(-2, 2.25, 0.25)
colors = plt.cm.PuOr_r(np.linspace(0, 1, len(intervals) - 1))  # Use YlOrRd colormap

# Create a custom colormap with specified intervals and colors
cmap = mcolors.ListedColormap(colors)
bounds = mcolors.BoundaryNorm(intervals, cmap.N)



In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-5, vcenter=0, vmax=1)

plot = (DCP-PRISM).tasmax.sel(quantile=0.9).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' , add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('90th Percentile  Max Temperature Difference [Degree C]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
cbar.set_label('Temperature Difference [Degree C]',fontweight='bold', fontsize=14)
plt.savefig('./plot/fig3_0.9_max_temp_dif.png')


In [ ]:
var="tasmin"
DCP_base=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/annual_average/"+var+"/2014.zarr")

datasets=[]
models=glob.glob("/nobackupp28/skhajehe/dcp-indices/multimodel/quantile/"+var+"/*_sort.zarr")
# List of file names
for model in models:
    X1=xr.open_zarr(model,consolidated=False)
    datasets.append(X1)
DCP = xr.concat(datasets, dim='model').mean(dim="model")
DCP=DCP.where(DCP_base[var].notnull())

PRISM=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/prism/quantile/"+var+"/quantiles.zarr")
PRISM=PRISM.where(DCP_base[var].notnull())

DCP['lon'] = (DCP['lon'] + 180) % 360 - 180
DCP = DCP.sortby(DCP.lon)
PRISM['lon'] = (PRISM['lon'] + 180) % 360 - 180
PRISM = PRISM.sortby(PRISM.lon)
DCP.load()
PRISM.load()

In [ ]:
DCP

In [ ]:
xr.open_zarr('/nobackupp28/skhajehe/dcp-indices/prism/quantile/tasmin/quantiles.zarr',consolidated=False)

In [ ]:
!ls /nobackupp28/skhajehe/dcp-indices/prism/quantile/tasmin

In [ ]:
DCP.tasmin.sel(quantile=0.1).max().values

In [ ]:
## import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Define color intervals and corresponding colors
intervals = np.arange(-18, 28, 4)
colors = plt.cm.cividis(np.linspace(0, 1, len(intervals) - 1))  # Use YlOrRd colormap

# Create a custom colormap with specified intervals and colors
cmap = mcolors.ListedColormap(colors)
bounds = mcolors.BoundaryNorm(intervals, cmap.N)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=20)

plot = DCP.tasmin.sel(quantile=0.25).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' ,add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('DCP30 Ensemble Mean 25th Percentile Min Temperature [Degree C]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

# cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
# cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
# cbar.set_label('Temperature [Degree C]',fontweight='bold', fontsize=14)
# cbar.ax.yaxis.set_tick_params(labelsize='large')
# plt.show()
plt.savefig('./plot/fig3_0.25_min_temp_dcp.png')


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=20)

plot = PRISM.tasmin.sel(quantile=0.5).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' ,add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('PRISM 50th Percentile Min Temperature [Degree C]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

# cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
# cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
# cbar.set_label('Temperature [Degree C]',fontweight='bold', fontsize=14)
# cbar.ax.yaxis.set_tick_params(labelsize='large')
# plt.show()
plt.savefig('./plot/fig3_0.5_min_temp_prism.png')


In [ ]:
## import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Define color intervals and corresponding colors
intervals = np.arange(-2, 2.25, 0.25)
colors = plt.cm.PuOr_r(np.linspace(0, 1, len(intervals) - 1))  # Use YlOrRd colormap

# Create a custom colormap with specified intervals and colors
cmap = mcolors.ListedColormap(colors)
bounds = mcolors.BoundaryNorm(intervals, cmap.N)



In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
usa_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_states[usa_states['continent'] == 'North America'].plot(ax=ax, edgecolor='none', facecolor='lightgrey', linewidth=0.8)
# cmap = mcolors.TwoSlopeNorm(vmin=-5, vcenter=0, vmax=1)

plot = (DCP-PRISM).tasmin.sel(quantile=0.9).plot(ax=ax,cmap=cmap, norm=bounds,levels=intervals, extend='both' , add_colorbar=False)
# Add state boundaries
states = gpd.read_file("conus.geojson")
states.boundary.plot(ax=ax, linewidth=0.8, color='black')
# Setting x and y axis limits
ax.set_xlim(-125, -65)  # Example values, adjust according to your data
ax.set_ylim(24, 50)  # Example values, adjust according to your data

plt.title('90th Percentile Min Temperature Difference [Degree C]', fontweight='bold',fontsize=16)
plt.xlabel("Longitude", fontsize=16)
plt.ylabel("Latitude", fontsize=16)

cax = fig.add_axes([0.92, 0.285, 0.02, 0.440])  # [left, bottom, width, height]
cbar = plt.colorbar(plot, cax=cax, orientation='vertical',extendfrac='auto')
cbar.set_label('Temperature Difference [Degree C]',fontweight='bold', fontsize=14)
plt.savefig('./plot/fig3_0.9_min_temp_dif.png')
